In [364]:
import socket
import numpy as np
import time
import pickle

In [365]:
#List of IP addresses for different FPGAs
HOST = ["0.0.0.0", "0.0.0.0", "0.0.0.0", "0.0.0.0"]
PORT = [65432, 65433, 65434, 65435]  # Port to listen on (non-privileged ports are > 1023)

In [366]:
#Randomly generating input arrays
arr1 = np.random.randint(1000, size = (100,150),dtype='i2')
arr2 = np.random.randint(1000, size = (150,200),dtype='i2')

In [367]:
#Flattening 2d Arrays to 1d
arr1_ser= arr1.flatten()
arr2_ser= arr2.flatten()

In [368]:
#Converting the arrays to send to bytes
arr1_bytes = np.array(arr1_ser, dtype=np.int16).tobytes()
arr2_bytes = np.array(arr2_ser, dtype=np.int16).tobytes()
#arr2_2_bytes = np.array(arr2_ser[15000:30000], dtype=np.int16).tobytes()

In [369]:
#Sending data to all the FPGAs one by one
for i in range(len(HOST)):
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST[i], PORT[i]))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")

            data= arr1_bytes
            conn.sendall(data)
            print("1st Array sent: ", arr1_ser)
            
            
            data= arr2_bytes
            conn.sendall(data)
            print("2nd Array sent: ", arr2_ser)
            
            
#            data= arr2_2_bytes
#            conn.sendall(data)
            
            

Connected by ('192.168.0.101', 33606)
1st Array sent:  [ 43 349 495 ... 181 862  95]
2nd Array sent:  [294 296 660 ... 247 391   1]


KeyboardInterrupt: 

# Create a socket object
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Define the server address and port
server_address = (HOST[0], PORT[0])

# Connect to the server
sock.connect(server_address)

# Generate an array of size 500,000 (example)
array_size = 500000
array = list(range(array_size))

# Serialize the array using pickle
serialized_array = pickle.dumps(array)

# Send the serialized array to the server
sock.sendall(serialized_array)

# Close the socket
sock.close()

In [ ]:
#Receiving the computed matrix from the FPGAs
ii=0
PORT= [6000, 6001, 6002, 6003]

fpga_res={}

for i in range(len(HOST)):
    
    arr=[]
    arr1=[]
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST[i], PORT[i]))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")            
            
            for j in range(100):
                result_bytes = conn.recv(2*200)
                result = np.frombuffer(result_bytes, dtype=np.int16)
                arr= result.copy()
                arr1.append(arr)
        
    arr1 = [x for y in arr1 for x in y]
    arr1 = np.array(list(arr1), dtype='int16')
    arr1= arr1.reshape(100,200)
    
    ii = ii + 1
    
    fpga_res[ii]= arr1[::]
    print("Array received from FPGA- "+str(ii)+" : ", fpga_res[ii])
            